In [25]:
import numpy as np
import pandas as pd
import csv

import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import *
from collections import Counter
from transformers import BertModel, BertTokenizer
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import torch

## 추천 시스템 모델링
1. 전처리
2. 토큰화, 벡터화(bert)
3. 사용자 입력 및 유사도 계산

In [26]:
#파일 가져오기
df_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/최종_데이터셋3(색상).csv"
stopword_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/불용어리스트_한국어.txt"

In [27]:
df = pd.read_csv(df_file, encoding='CP949')
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head(5)

,꽃,월,계절,꽃말,설명,이미지,색상
0,수염패랭이꽃,6,여름,의협심,"투쟁심, 정의감, 미적 센스가 풍부한 사람이지만 의사표현이 풍부한 사람입니다. 그 ...",https://www.picturethisai.com/image-handle/web...,분홍
1,쉬라즈 장미,6,여름,낭만,요즘 겹겹이 꽃잎이 쌓인 화형이 유행처럼 많이 생산하고 있지만 쉬라즈 장미는 나팔처...,https://file.honestflower.kr/media/images/ingr...,주황
2,에린지움,8,여름,비밀스런 애정,자기 표현에 서투르며 쓸쓸해 하고 고독을 사랑하는 당신. 좀 더 당신의 기분을 개방...,https://blog.kakaocdn.net/dn/Li1Dq/btrec3wNAnF...,파랑
3,꽃담배,8,여름,그대있어 외롭지 않네,당신만 있다면 인생은 장미빛이라고 믿는 누군가가 있다면 꼭 붙잡으세요.,https://encrypted-tbn0.gstatic.com/images?q=tb...,분홍
4,슈퍼센세이션 장미,6,여름,행복한 사랑,동글동글한 화형으로 드라이가 된 후에도 변형이 적어 드라이 플라워로 많이 쓰이는 스...,https://file.honestflower.kr/media/images/ingr...,분홍


In [28]:
#nlp처리 할 데이터만 가져오기
df_nlp = df[["꽃", "꽃말", "설명"]]
print(df_nlp.shape)
df_nlp.head(5)

(824, 3)


,꽃,꽃말,설명
0,수염패랭이꽃,의협심,"투쟁심, 정의감, 미적 센스가 풍부한 사람이지만 의사표현이 풍부한 사람입니다. 그 ..."
1,쉬라즈 장미,낭만,요즘 겹겹이 꽃잎이 쌓인 화형이 유행처럼 많이 생산하고 있지만 쉬라즈 장미는 나팔처...
2,에린지움,비밀스런 애정,자기 표현에 서투르며 쓸쓸해 하고 고독을 사랑하는 당신. 좀 더 당신의 기분을 개방...
3,꽃담배,그대있어 외롭지 않네,당신만 있다면 인생은 장미빛이라고 믿는 누군가가 있다면 꼭 붙잡으세요.
4,슈퍼센세이션 장미,행복한 사랑,동글동글한 화형으로 드라이가 된 후에도 변형이 적어 드라이 플라워로 많이 쓰이는 스...


In [29]:
#꽃말과 설명 합치기
df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['설명']
df_nlp.drop("꽃말", axis=1, inplace=True)
df_nlp.head(5)

C:\Users\pc\AppData\Local\Temp\ipykernel_23976\3604583926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['설명']
C:\Users\pc\AppData\Local\Temp\ipykernel_23976\3604583926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp.drop("꽃말", axis=1, inplace=True)


,꽃,설명
0,수염패랭이꽃,"의협심. 투쟁심, 정의감, 미적 센스가 풍부한 사람이지만 의사표현이 풍부한 사람입니..."
1,쉬라즈 장미,낭만. 요즘 겹겹이 꽃잎이 쌓인 화형이 유행처럼 많이 생산하고 있지만 쉬라즈 장미는...
2,에린지움,비밀스런 애정. 자기 표현에 서투르며 쓸쓸해 하고 고독을 사랑하는 당신. 좀 더 당...
3,꽃담배,그대있어 외롭지 않네. 당신만 있다면 인생은 장미빛이라고 믿는 누군가가 있다면 꼭 ...
4,슈퍼센세이션 장미,행복한 사랑. 동글동글한 화형으로 드라이가 된 후에도 변형이 적어 드라이 플라워로 ...


#### 1. 전처리

In [30]:
#특수문자 및 띄어쓰기 제거
import re

def remove_special_characters(text):
    # 특수문자 제거
    pattern = r'[^\w\s\.]' #문자,공백문자,마침표 제외 제거
    clean_text = re.sub(pattern, '', text)

    # 한자 제거
    pattern = r'[\u4e00-\u9fff]' #중국어 한자의 유니코드 시작과 끝 제거
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = ' '.join(clean_text.split())
    return clean_text

df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
df_nlp.head(10)

C:\Users\pc\AppData\Local\Temp\ipykernel_23976\1447322065.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)


,꽃,설명
0,수염패랭이꽃,의협심. 투쟁심 정의감 미적 센스가 풍부한 사람이지만 의사표현이 풍부한 사람입니다....
1,쉬라즈 장미,낭만. 요즘 겹겹이 꽃잎이 쌓인 화형이 유행처럼 많이 생산하고 있지만 쉬라즈 장미는...
2,에린지움,비밀스런 애정. 자기 표현에 서투르며 쓸쓸해 하고 고독을 사랑하는 당신. 좀 더 당...
3,꽃담배,그대있어 외롭지 않네. 당신만 있다면 인생은 장미빛이라고 믿는 누군가가 있다면 꼭 ...
4,슈퍼센세이션 장미,행복한 사랑. 동글동글한 화형으로 드라이가 된 후에도 변형이 적어 드라이 플라워로 ...
5,스노우플레이크 장미,순수한 사랑. 결혼식. 이름처럼 새하얀 눈송이를 양껏 모아놓은 스프레이 장미예요. ...
6,스위트니스 장미,사랑. 꽃잎의 대부분은 눈처럼 하얀색이지만 꽃잎의 끝면은 선명한 붉은 색으로 강렬한...
7,스위트피,우아한 추억. 정이 많고 헌신적인 사람이지만 상대를 잘 파악하고 자신을 지키는 것도...
8,스윗유니크 장미,사랑의 맹세. 성년의 날 발렌타인데이 로즈데이. 핑크 하면 떠오르는 그 색감 꽃잎 ...
9,스윗핑크 장미,영원한 사랑. 동글동글 알사탕같이 귀여운 매력을 담은 스윗핑크 장미는 타 스프레이 ...


#### 2. 토큰화, 벡터화(bert)
[토큰화]
- bert의 토큰화 진행 시, BERT의 설계와 사전 학습된 방식에 맞추어 불용어 제거 없이 사용하는 것이 일반적으로 더 나은 성능을 발휘 </br>

[벡터화]
- bert를 통해 '꽃말', '설명' 칼럼 벡터 추출 -> 하나의 벡터로 결합
- 월, 계절 원핫인코딩 -> 하나의 벡터
- 총 2개의 벡터 생성 </br>

*문장 벡터의 차원이 훨씬 커서 원핫 인코딩 벡터의 상대적인 중요성이 낮아질 수 있기 때문

불용어 리스트: https://wikidocs.net/22530

In [31]:
# BERT 모델과 토크나이저 불러오기
model_name = 'bert-base-multilingual-cased' # 다국어 전용 / 한국어 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [32]:
# # 문장을 토큰화합니다.
# tokens = tokenizer.tokenize(df_nlp['설명'][0])
# print("Tokens:", tokens)

# # 토큰을 ID로 변환합니다.
# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# print("Token IDs:", token_ids)

# outputs = model(**inputs)

# print(outputs.last_hidden_state.shape, pooler_output) #(1,190,768) -> 190개의 단어 / (1,768)

In [33]:
# [CLS]와 [SEP] 토큰을 추가하는 함수
def add_special_tokens(text):
    # 문장을 마침표를 기준으로 분리
    sentences = text.split('. ')
    # 각 문장에 [CLS]와 [SEP] 토큰 추가
    sentences_with_special_tokens = ['[CLS] ' + sentence + ' [SEP]' for sentence in sentences]
    # 다시 문장으로 결합
    text_with_special_tokens = ' '.join(sentences_with_special_tokens)
    return text_with_special_tokens

In [34]:
df_nlp['설명2'] = df_nlp['설명'].apply(add_special_tokens)
df_nlp.head(10)

C:\Users\pc\AppData\Local\Temp\ipykernel_23976\3322456421.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명2'] = df_nlp['설명'].apply(add_special_tokens)


,꽃,설명,설명2
0,수염패랭이꽃,의협심. 투쟁심 정의감 미적 센스가 풍부한 사람이지만 의사표현이 풍부한 사람입니다....,[CLS] 의협심 [SEP] [CLS] 투쟁심 정의감 미적 센스가 풍부한 사람이지만...
1,쉬라즈 장미,낭만. 요즘 겹겹이 꽃잎이 쌓인 화형이 유행처럼 많이 생산하고 있지만 쉬라즈 장미는...,[CLS] 낭만 [SEP] [CLS] 요즘 겹겹이 꽃잎이 쌓인 화형이 유행처럼 많이...
2,에린지움,비밀스런 애정. 자기 표현에 서투르며 쓸쓸해 하고 고독을 사랑하는 당신. 좀 더 당...,[CLS] 비밀스런 애정 [SEP] [CLS] 자기 표현에 서투르며 쓸쓸해 하고 고...
3,꽃담배,그대있어 외롭지 않네. 당신만 있다면 인생은 장미빛이라고 믿는 누군가가 있다면 꼭 ...,[CLS] 그대있어 외롭지 않네 [SEP] [CLS] 당신만 있다면 인생은 장미빛이...
4,슈퍼센세이션 장미,행복한 사랑. 동글동글한 화형으로 드라이가 된 후에도 변형이 적어 드라이 플라워로 ...,[CLS] 행복한 사랑 [SEP] [CLS] 동글동글한 화형으로 드라이가 된 후에도...
5,스노우플레이크 장미,순수한 사랑. 결혼식. 이름처럼 새하얀 눈송이를 양껏 모아놓은 스프레이 장미예요. ...,[CLS] 순수한 사랑 [SEP] [CLS] 결혼식 [SEP] [CLS] 이름처럼 ...
6,스위트니스 장미,사랑. 꽃잎의 대부분은 눈처럼 하얀색이지만 꽃잎의 끝면은 선명한 붉은 색으로 강렬한...,[CLS] 사랑 [SEP] [CLS] 꽃잎의 대부분은 눈처럼 하얀색이지만 꽃잎의 끝...
7,스위트피,우아한 추억. 정이 많고 헌신적인 사람이지만 상대를 잘 파악하고 자신을 지키는 것도...,[CLS] 우아한 추억 [SEP] [CLS] 정이 많고 헌신적인 사람이지만 상대를 ...
8,스윗유니크 장미,사랑의 맹세. 성년의 날 발렌타인데이 로즈데이. 핑크 하면 떠오르는 그 색감 꽃잎 ...,[CLS] 사랑의 맹세 [SEP] [CLS] 성년의 날 발렌타인데이 로즈데이 [SE...
9,스윗핑크 장미,영원한 사랑. 동글동글 알사탕같이 귀여운 매력을 담은 스윗핑크 장미는 타 스프레이 ...,[CLS] 영원한 사랑 [SEP] [CLS] 동글동글 알사탕같이 귀여운 매력을 담은...


In [35]:
# 문장을 벡터화 변환하는 함수
def get_sentence_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True) #최대길이 초과 시 잘라내기, 작은 경우 패딩진행
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy() #각 토큰 벡터의 첫 번째 벡터 확인 (CLS 토큰 벡터)

[CLS] 토큰 벡터는 문장의 전체 의미를 포착하며, 서브워드 "용", "##기"의 벡터들은 각각의 서브워드 의미를 포함하여 문맥 정보를 반영함

In [36]:
#설명 벡터화
df_nlp['설명_벡터'] = df_nlp['설명2'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist()) #데이터마다 (1, 768) 차원 벡터 생성

C:\Users\pc\AppData\Local\Temp\ipykernel_23976\1200080225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명_벡터'] = df_nlp['설명2'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist()) #데이터마다 (1, 768) 차원 벡터 생성


In [37]:
# type(df_nlp['설명_벡터'][1])
# pd.DataFrame(df_nlp['설명_벡터'][1])

In [40]:
#월과 계절 원핫인코딩
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[['월', '계절']]).toarray()  # '월'과 '계절' 컬럼을 원핫인코딩
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['월', '계절'])) # 원핫인코딩된 결과를 새로운 데이터프레임으로 변환

# 원핫인코딩 결과를 '원핫인코딩' 컬럼으로 추가
df_nlp['원핫인코딩'] = encoded_df.apply(lambda row: list(row), axis=1)
df_nlp['원핫인코딩'] = df_nlp['원핫인코딩'].apply(lambda x: np.array(x).reshape(1, 16)) #(16,1)을 (1,16)차원으로 변경

C:\Users\pc\AppData\Local\Temp\ipykernel_23976\2934671132.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['원핫인코딩'] = encoded_df.apply(lambda row: list(row), axis=1)
C:\Users\pc\AppData\Local\Temp\ipykernel_23976\2934671132.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['원핫인코딩'] = df_nlp['원핫인코딩'].apply(lambda x: np.array(x).reshape(1, 16)) #(16,1)을 (1,16)차원으로 변경


In [41]:
# 결합 벡터 생성 (설명 벡터, 꽃말 벡터 결합)
def create_combined_vector(row):
    설명_벡터 = row['설명_벡터'] #(1,768)
    원핫인코딩 = row['원핫인코딩'] #(1,16)

    combined_array = np.concatenate((설명_벡터, 원핫인코딩), axis=1) #(1,784) 

    return combined_array

df_nlp['최종_벡터'] = df_nlp.apply(create_combined_vector, axis=1)

C:\Users\pc\AppData\Local\Temp\ipykernel_23976\2624279429.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['최종_벡터'] = df_nlp.apply(create_combined_vector, axis=1)


In [ ]:
# with open(stopword_file, 'r', encoding='utf-8') as file:
#     stopwords = file.read() # 파일 내용 읽기
# print(stopwords)

# #트큰화 (불용어 제외) - 빈도수 파악 위해
# okt = Okt()

# word_list=[]
# for i in df_nlp['설명']:
#     word_tokens = okt.morphs(i) #형태소 분석

#     for j in word_tokens:
#         if j not in stopwords: 
#             word_list.append(j)
            
# word_list

# #빈도수 확인
# frequent = Counter(word_list).most_common()
# frequent[:20]

# #불용어 재구축
# stopwords_re = "꽃 입니다 줄기 있는 이나 품종 처럼 식물 많이 에서는"
# stopwords_re  = stopwords_re +' ' + stopwords
# stopwords_re = stopwords_re.replace('의지하여', '')
# stopwords_re

In [ ]:
# #불용어 제거 함수화
# def remove_stopword(text):
#     word_tokens = okt.morphs(text) #토큰화
    
#     a= []
#     for i in word_tokens:
#         if i not in stopwords_re: #불용어가 아닌 단어만 넣기
#             a.append(i)

#     return a

# #불용어가 제거된 칼럼 생성
# df_nlp['설명_불용어제거'] = df_nlp['설명'].apply(remove_stopword)
# df_nlp.head(5)

# #형용사, 명사 등만 남기고 불용어 제거
# def extract_nouns_adjectives(text):
#     pos_tagged = okt.pos(text) #형태소 추출

#     nouns = [word for word, pos in pos_tagged if pos == 'Noun']
#     adjectives = [word for word, pos in pos_tagged if pos == 'Adjective']
#     a = nouns + adjectives
   
#     a_re=[]
#     for i in a:
#         if i not in stopwords_re: 
#             a_re.append(i)
#     return a_re

# #불용어 제거 및 형용사, 명사 형태소만 남긴 칼럼 생성
# df_nlp['설명_명사,형용사'] = df_nlp['설명'].apply(extract_nouns_adjectives)
# df_nlp[['설명', '설명_불용어제거', '설명_명사,형용사']].head(5)

# #트큰화 (불용어 제외, 명사와 형용사만 추출) - 빈도수 파악 위해
# word_list= []
# for i in df_nlp['설명_명사,형용사']:
#     for j in i:
#         if j not in stopwords_re: #불용어가 아닌 것만 넣기
#             word_list.append(j)

# #빈도수 확인
# frequent = Counter(word_list).most_common()
# frequent[:20]

# #불용어 재구축
# stopwords_re2 = "좋은 이름 연출 대표 사용 있습니다"
# stopwords_re  = stopwords_re2 +' ' + stopwords_re
# stopwords_re

# #불용어 제거 및 형용사, 명사 형태소만 남긴 칼럼 재생성 (위 불용어 적용하여)
# df_nlp['설명_명사,형용사'] = df_nlp['설명'].apply(extract_nouns_adjectives)
# df_nlp[['설명', '설명_불용어제거', '설명_명사,형용사']].head(5)

# #리스트 내에 중복되는 부분 제거
# from collections import OrderedDict

# df_nlp['설명_명사,형용사'] = df_nlp['설명_명사,형용사'].apply(lambda x: list(OrderedDict.fromkeys(x))) #순서 섞이지 않도록 고정
# df_nlp[['설명', '설명_불용어제거', '설명_명사,형용사']].head(5)

In [ ]:
# # sentencepiece로 토큰화하는 방법

# with open('flower_content.txt', 'w', encoding='utf8') as f:
#     f.write('\n'.join(df_nlp['설명']))

# pip install sentencepiece

# import sentencepiece as spm
# spm.SentencePieceTrainer.Train('--input=flower_content.txt --model_prefix=flower --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

# vocab_list = pd.read_csv('flower.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
# sp = spm.SentencePieceProcessor()
# vocab_file = "flower.model"
# sp.load(vocab_file)

# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # 문장을 토큰화,벡터화 변환하는 함수
# def get_sentence_embedding(text):
#     token_ids = sp.encode_as_ids(text) #사전 번호와 맵핑
#     token_ids = [token_ids]
#     encoded = pad_sequences(encoded, maxlen=340, padding='post', truncating='post')
#     encoded_vector = np.array(encoded)

#     return encoded_vector

# #설명 벡터화
# df_nlp['설명_벡터'] = df_nlp['설명'].apply(lambda x: get_sentence_embedding(x)) 

#### 4. 사용자 입력 및 유사도 계산 
- 사용자 입력 벡터화
- 유사도 계산

 > 사용자 입력에서 월이나 계절 내용이 없으면 사용자가 입력한 날짜를 추출해서 가져오기

In [42]:
#꽃말 칼럼 생성
df_nlp['꽃말'] = df['꽃말']

In [43]:
df_nlp.head(5)

,꽃,설명,설명2,설명_벡터,원핫인코딩,최종_벡터,꽃말
0,수염패랭이꽃,의협심. 투쟁심 정의감 미적 센스가 풍부한 사람이지만 의사표현이 풍부한 사람입니다....,[CLS] 의협심 [SEP] [CLS] 투쟁심 정의감 미적 센스가 풍부한 사람이지만...,"[[-0.1247178316116333, 0.06621979176998138, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[-0.1247178316116333, 0.06621979176998138, -0...",의협심
1,쉬라즈 장미,낭만. 요즘 겹겹이 꽃잎이 쌓인 화형이 유행처럼 많이 생산하고 있지만 쉬라즈 장미는...,[CLS] 낭만 [SEP] [CLS] 요즘 겹겹이 꽃잎이 쌓인 화형이 유행처럼 많이...,"[[0.19904625415802002, 0.06422638893127441, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[0.19904625415802002, 0.06422638893127441, -0...",낭만
2,에린지움,비밀스런 애정. 자기 표현에 서투르며 쓸쓸해 하고 고독을 사랑하는 당신. 좀 더 당...,[CLS] 비밀스런 애정 [SEP] [CLS] 자기 표현에 서투르며 쓸쓸해 하고 고...,"[[0.15861323475837708, 0.05082745850086212, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[0.15861323475837708, 0.05082745850086212, -0...",비밀스런 애정
3,꽃담배,그대있어 외롭지 않네. 당신만 있다면 인생은 장미빛이라고 믿는 누군가가 있다면 꼭 ...,[CLS] 그대있어 외롭지 않네 [SEP] [CLS] 당신만 있다면 인생은 장미빛이...,"[[0.026733960956335068, 0.08796659111976624, -...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[0.026733960956335068, 0.08796659111976624, -...",그대있어 외롭지 않네
4,슈퍼센세이션 장미,행복한 사랑. 동글동글한 화형으로 드라이가 된 후에도 변형이 적어 드라이 플라워로 ...,[CLS] 행복한 사랑 [SEP] [CLS] 동글동글한 화형으로 드라이가 된 후에도...,"[[0.10028460621833801, 0.06559312343597412, -0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[[0.10028460621833801, 0.06559312343597412, -0...",행복한 사랑


In [44]:
# 사용자 입력 텍스트 분석 함수(월, 계절)
def extract_month_season(text):
    months = {
        '1월': 1, '2월': 2, '3월': 3, '4월': 4, '5월': 5, '6월': 6,
        '7월': 7, '8월': 8, '9월': 9, '10월': 10, '11월': 11, '12월': 12
    }
    seasons = {'봄': '봄', '여름': '여름', '가을': '가을', '겨울': '겨울'}
    
    month = None
    season = None
    
    for key, value in months.items():
        if key in text:
            month = value
            break
    
    for key in seasons.keys():
        if key in text:
            season = key
            break
    
    return month, season

In [45]:
#사용자 입력을 최종 벡터화 
def get_user_input_vector(user_input):
    input = remove_special_characters(user_input) #특수문자 제거
    
    # 토큰화 및 벡터화
    input_embeddings = get_sentence_embedding(input, tokenizer, model) #(1,768)
    month, season = extract_month_season(user_input)

    # 원핫 벡터 생성
    user_onehot_vector = np.zeros(len(encoder.get_feature_names_out(['월', '계절'])))
    if month is not None:
        month_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'월_{month}')
        user_onehot_vector[month_idx] = 1
    if season is not None:
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[season_idx] = 1
    user_onehot_vector = user_onehot_vector.reshape(1, 16) #(1,16)

    #벡터 결합
    user_vector = np.concatenate((input_embeddings, user_onehot_vector), axis=1) #(1,784)
    return user_vector

In [46]:
# 추천 시스템 함수 (코사인 유사도 기반)
from sklearn.metrics.pairwise import cosine_similarity

def recommend_flower(user_input, tokenizer, model):
    user_vector = get_user_input_vector(user_input) #사용자 입력을 최종 벡터화

    #코사인 유사도 산출
    df_nlp['유사도'] = df_nlp['최종_벡터'].apply(lambda x: cosine_similarity(user_vector, x)[0][0])
    df_nlp['유사도'] = df_nlp['유사도'].astype(float) #숫자형으로 변환

    # 유사도를 기준으로 상위 3개의 꽃을 선택하고 중복된 꽃을 제거
    top3 = df_nlp.nlargest(3, '유사도').drop_duplicates(subset='꽃')

    # 만약 중복 제거 후 3개의 꽃이 되지 않는 경우, 다시 nlargest로 채우기
    if top3.shape[0] < 3:
        additional_top = df_nlp.nlargest(20, '유사도')  # 상위 10개 정도를 선택
        additional_top = additional_top[~additional_top['꽃'].isin(top3['꽃'])]
        top3 = pd.concat([top3, additional_top]).nlargest(3, '유사도').drop_duplicates(subset='꽃')

    return top3[['꽃', '꽃말', '유사도']]

In [47]:
#성능 테스트 해보기

#"여자친구 생일이 6월 10일이야"
#"친구가 어제 드디어 취업을 해서 꽃을 선물하려해"
#"어제 여자친구랑 싸워서 화해를 하려해"
#"부모님 생신이여서 꽃을 선물하고 싶은데 화려한 꽃으로 축하하고 싶어"
#"어버이날"
#"나는 1월에 겨울에 어울리는 용기 있는 꽃을 찾고 있어"

user_input = "여자친구 생일이 6월 10일이야"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,꽃말,유사도
110,푸에고 장미,상냥함,0.910202
122,피칸토 장미,귀여운 여인,0.907399
97,트윙클브라이드 장미,고마움,0.905882


모두 6월 여름으로 추출됨 </br>
피칸토 장미(귀여운 여인) / 트윙클브라이드 장미(빛나는 여자)

In [48]:
user_input = "친구가 어제 드디어 취업을 해서 꽃을 선물하려해"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,꽃말,유사도
650,데이지,진실하 사랑,0.968230
575,꽃 영춘화,사모하는 마음,0.967570
184,자이언트 델피늄,제 마음을 헤아려 주세요,0.965653


취업, 선물, 친구라는 단어가 존재하지 않아서 그런지 잘 모르겠음

In [49]:
user_input = "어제 여자친구랑 싸워서 화해를 하려해"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,꽃말,유사도
18,시계꽃,성스러운 사랑,0.953387
16,스프레이 델피늄,당신을 행복하게 해줄게요,0.951887
456,네모필라,애국심,0.951516


모두 사랑이라는 의미를 갖고 있긴 함

In [50]:
user_input = "부모님 생신이여서 꽃을 선물하고 싶은데 화려한 꽃으로 축하하고 싶어"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,꽃말,유사도
163,세일러문 해바라기,"숭배, 동경",0.961955
597,복숭아 꽃,"번영, 풍요, 행복",0.959871
486,오니소갈륨,일편단심,0.958699


해바라기에 동경, 쉼머 장미는 사랑과 화사함, 복숭아 꽃은 풍요와 행복과 화사함

In [51]:
user_input = "어버이날"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,꽃말,유사도
763,리나 카네이션,사랑과 존경,0.902778
10,스칼렛플러스 카네이션,사랑과 존경,0.896615
11,스타 체리테시노 카네이션,사랑과 존경,0.896615


In [52]:
# 예시 사용자 입력
user_input = "나는 1월에 겨울에 어울리는 용기 있는 꽃을 찾고 있어"

# 꽃 추천
recommend_flower(user_input, tokenizer, model)

,꽃,꽃말,유사도
446,패럿 튤립,사랑의 고백,0.951115
502,겹설유화,애교,0.950162
449,히아신스,"유희, 겸손한 사랑",0.949900


패럿 튤립 1월(사랑), 겹설유화(겨울), 히아신스(용기재배..)